# Load vector DB collection

In [ ]:
import chromadb
    
client = chromadb.PersistentClient(path="../data/vector_db/")

In [21]:
def get_chroma_db(db_name: str):
    """Get a ChromaDB collection by name"""
    return client.get_collection(name=db_name)

In [10]:
collection = client.get_or_create_collection(
    name="world_history",
    metadata={"hnsw:space": "cosine"},
)

In [4]:
collection.count()

7353

## Query the vector db

In [1]:
results = collection.query(
    query_texts=["what were the key features of Sumerian city-states?"],
    n_results=10,
)

print(results)

NameError: name 'collection' is not defined

In [22]:
def query_the_db(query, collection, n_results):
    """Query the ChromaDB collection with a question"""
    results = collection.query(
        query_texts=[query],  # Must be a list
        n_results=n_results
    )
    return results

# Using Gemini API as the LLM model

In [2]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=api_key)

In [3]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash

# Prompting gemini model

In [12]:
def make_prompt(query, data):
    """
    Build the strict ChromaDB retrieval prompt.
    'query'  = the user question
    'data'   = the retrieved chunks from ChromaDB (result from collection.query())
    """
    
    # Extract the documents from the ChromaDB results
    # data['documents'] is a list of lists, so we take the first list
    documents = data.get('documents', [[]])[0]
    
    # Format the documents as numbered context items
    context_text = "\n".join([f"{i+1}. {doc}" for i, doc in enumerate(documents)])

    prompt = f"""
ROLE: Chroma Retrieval Executor

PRIMARY DIRECTIVE:
Use only the information contained inside the DELIMITED CONTEXT BLOCK to answer the query.
Base every part of your response solely on those context details.

CONSTRAINTS (MANDATORY):
- Maintain a zero-hallucination standard.
- When the context supports the answer, state it directly and concisely.
- When the context lacks the required information, respond exactly:
  "Not found in ChromaDB."
- Produce only the final answer with no additional commentary.

OPERATIONAL RULES:
1. Treat the DELIMITED CONTEXT BLOCK as the authoritative source.
2. Ground every statement directly in the context.
3. Respond only when the context provides explicit support.
4. Keep the answer minimal, precise, and context-bound.

===== DELIMITED CONTEXT BLOCK START =====
{context_text}
===== DELIMITED CONTEXT BLOCK END =====

USER QUERY:
{query}

RESPONSE POLICY:
- If the context contains the answer → return it.
- If the context does not contain the answer → return: Not found in ChromaDB.
"""
    return prompt


In [13]:
# Test the prompt function
query = "what were the key features of Sumerian city-states?"
prompt = make_prompt(query, results)

print(prompt)


ROLE: Chroma Retrieval Executor

PRIMARY DIRECTIVE:
Use only the information contained inside the DELIMITED CONTEXT BLOCK to answer the query.
Base every part of your response solely on those context details.

CONSTRAINTS (MANDATORY):
- Maintain a zero-hallucination standard.
- When the context supports the answer, state it directly and concisely.
- When the context lacks the required information, respond exactly:
  "Not found in ChromaDB."
- Produce only the final answer with no additional commentary.

OPERATIONAL RULES:
1. Treat the DELIMITED CONTEXT BLOCK as the authoritative source.
2. Ground every statement directly in the context.
3. Respond only when the context provides explicit support.
4. Keep the answer minimal, precise, and context-bound.

===== DELIMITED CONTEXT BLOCK START =====
1. Sumerian cities had certain characteristics in common.
2. Appearance of Sumerian city-states in lower Mesopotamia.
3. While Sumerian traditions influenced developments throughout the region, o

# Generating the response

## Getting the model

In [16]:
model = genai.GenerativeModel('models/gemini-2.5-pro')

## Prompting the model

In [ ]:
answer = model.generate_content(prompt)

NameError: name 'Markdown' is not defined

In [18]:
from IPython.display import Markdown

Markdown(answer.text)

Sumerian city-states were the basic organizational structure of Sumerian civilization. The cities had organized religions, centralized governments, social hierarchies, and access to trade networks. They were divided into neighborhoods by occupation and kinship groups. Power was held by hereditary kings who led armies, collected taxes, organized labor for state projects, and meted out justice. These kings were legitimized through their association with gods. The Sumerians were also innovators with some of the first systems of writing and notable achievements in engineering and architecture.

# Whole pipeline

In [28]:
import time

# Step 1
question = "How was the culture like in ancient egypt?"

# Step 2
db = get_chroma_db("world_history")
data = query_the_db(question, db, n_results=10)

# Step 3
prompt = make_prompt(question, data)

# Step 4
model = genai.GenerativeModel('models/gemini-2.5-pro')

# Step 5 - Generate with retry logic
max_retries = 3
for attempt in range(max_retries):
    try:
        answer = model.generate_content(prompt)
        break
    except Exception as e:
        if "ResourceExhausted" in str(e) and attempt < max_retries - 1:
            wait_time = (attempt + 1) * 10  # Wait 10, 20, 30 seconds
            print(f"Rate limit hit. Waiting {wait_time} seconds...")
            time.sleep(wait_time)
        else:
            raise

# Step 6
from IPython.display import Markdown
Markdown(answer.text)

The ancient Egyptians created a vibrant civilization, while also finding comfort in the familiar and traditional. It was a complex society with sophisticated political and religious developments. Egypt produced agricultural surpluses that supported elites, priests, and skilled craftspeople, who made bureaucratic, religious, and artistic contributions. Egyptian innovations included stone-carving techniques, hieroglyphics, the use of papyrus, knowledge of the length of a solar year, and construction methods. Their culture was also influential on others, such as the Nubians.